## Training a differentially private LSTM model for name classification

In this tutorial we will build a differentially-private LSTM model to classify names to their source languages, which is the same task as in the tutorial **NLP From Scratch** (https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html). Since the objective of this tutorial is to demonstrate the effective use of an LSTM with privacy guarantees, we will be utilizing it in place of the bare-bones RNN model defined in the original tutorial. Specifically, we use the `DPLSTM` module from `opacus.layers.dp_lstm` to facilitate calculation of the per-example gradients, which are utilized in the addition of noise during application of differential privacy. `DPLSTM` has the same API and functionality as the `nn.LSTM`, with some restrictions (ex. we currently support single layers, the full list is given below).  

## Dataset

First, let us download the dataset of names and their associated language labels as given in https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html. We train our differentially-private LSTM on the same dataset as in that tutorial.

In [83]:
import requests

NAMES_DATASET_URL = "https://download.pytorch.org/tutorial/data.zip"
DATA_DIR = "names"

import zipfile
import urllib

def download_and_extract(dataset_url, data_dir):
    print("Downloading and extracting ...")
    filename = "data.zip"

    urllib.request.urlretrieve(dataset_url, filename)
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

download_and_extract(NAMES_DATASET_URL, DATA_DIR)

Completed!


In [84]:
import os

names_folder = os.path.join(DATA_DIR, 'data', 'names')
all_filenames = []

for language_file in os.listdir(names_folder):
    all_filenames.append(os.path.join(names_folder, language_file))
    
print(os.listdir(names_folder))

['Arabic.txt', 'Chinese.txt', 'Czech.txt', 'Dutch.txt', 'English.txt', 'French.txt', 'German.txt', 'Greek.txt', 'Irish.txt', 'Italian.txt', 'Japanese.txt', 'Korean.txt', 'Polish.txt', 'Portuguese.txt', 'Russian.txt', 'Scottish.txt', 'Spanish.txt', 'Vietnamese.txt']


We define the functions `unicode_to_ascii()` and `build_category_lines()` in the cell below. `unicode_to_ascii()` reads in a string and normalizes all characters in it to ASCII, removing all other Unicode characters. `read_lines()` reads the names dataset from disk and then stores it in the variable `category_lines` which is a dict with key as language and value as list of names belonging to that language. `all_categories` is a list of supported languages, and `n_categories` is the number of languages. 

In [85]:
import string 
import unicodedata

all_letters = string.ascii_letters + " .,;'#"
n_letters = len(all_letters)

def unicode_to_ascii(s, all_letters):
    return "".join(
        c
        for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn" and c in all_letters
    )

def read_lines(filename, all_letters):
    with open(filename) as f_read:
        lines = f_read.read().strip().split("\n")
    return [unicode_to_ascii(line, all_letters) for line in lines]

category_lines = {}
all_categories = []

for filename in all_filenames:
    category = filename.split("/")[-1].split(".")[0]
    all_categories.append(category)
    lines = read_lines(filename, all_letters)
    category_lines[category] = lines

n_categories = len(all_categories)

We inspect the dictionary `all_categories` and for each language key, see a few representatives names in each. For each language, we also find the number of names in it.

In [86]:
for language, names in category_lines.items():
    print(f"Language : {language} , names = {names[:5]}, # of names = {len(names)}")

Language : Arabic , names = ['Khoury', 'Nahas', 'Daher', 'Gerges', 'Nazari'], # of names = 2000
Language : Chinese , names = ['Ang', 'AuYong', 'Bai', 'Ban', 'Bao'], # of names = 268
Language : Czech , names = ['Abl', 'Adsit', 'Ajdrna', 'Alt', 'Antonowitsch'], # of names = 519
Language : Dutch , names = ['Aalsburg', 'Aalst', 'Aarle', 'Achteren', 'Achthoven'], # of names = 297
Language : English , names = ['Abbas', 'Abbey', 'Abbott', 'Abdi', 'Abel'], # of names = 3668
Language : French , names = ['Abel', 'Abraham', 'Adam', 'Albert', 'Allard'], # of names = 277
Language : German , names = ['Abbing', 'Abel', 'Abeln', 'Abt', 'Achilles'], # of names = 724
Language : Greek , names = ['Adamidis', 'Adamou', 'Agelakos', 'Akrivopoulos', 'Alexandropoulos'], # of names = 203
Language : Irish , names = ['Adam', 'Ahearn', 'Aodh', 'Aodha', 'Aonghuis'], # of names = 232
Language : Italian , names = ['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni'], # of names = 709
Language : Japane

## Training / Validation Set Preparation

We split the dataset into a 80-20 split for training and validation. We stratify across languages and for each language, we randomly choose 80% of the names and insert them into the training set, the remainder into the validation set. Note that the training and validation sets are also in the same format as `category_lines`

In [87]:
import random

category_lines_train = {}
category_lines_eval = {}
for key in category_lines.keys():
    category_lines_train[key] = []
    category_lines_eval[key] = []
for key in category_lines.keys():
    for val in category_lines[key]:
        if random.uniform(0, 1) < 0.8:
            category_lines_train[key].append(val)
        else:
            category_lines_eval[key].append(val)
            
dataset_size = sum(len(category_lines_train[key]) for key in category_lines_train.keys())

After splitting the dataset into a training and a validation set, we now have to convert the data into a numeric form suitable for training the LSTM model. For each name, we set a maximum sequence length of 15, and if a name is longer than the threshold, we truncate it (this rarely happens this dataset !). If a name is smaller than the threshold, we add a dummy `#` character to pad it to the desired length. We also batch the names in the dataset and set a batch size of 256 for all the experiments in this tutorial. The function `line_to_tensor()` returns a tensor of shape [15, 256] where each element is the index (in `all_letters`) of the corresponding character.

In [88]:
max_seq_length = 15
max_seq_length = 15

def line_to_tensor(batch_size, max_seq_length, lines, all_letters, n_letters):
    r"""
    Turns a list of batch_size lines into a <line_length x batch_size> tensor
    where each element of tensor is index of corresponding letter in all_letters
    """
    tensor = torch.zeros(max_seq_length, batch_size).type(torch.LongTensor)
    for batch_idx, line in enumerate(lines):
        # Pad/truncate line to fit to max_seq_length
        padded_line = line[0:max_seq_length] + "#" * (max_seq_length - len(line))
        for li, letter in enumerate(padded_line):
            letter_index = all_letters.find(letter)
            tensor[li][batch_idx] = letter_index
    return tensor

Before going to the model definition and attaching the privacy engine to the optimizer, we also define a few other functions for helping us fetch the batches. During training, we just draw random batches from the dataset, just like how it was done in the tutorial https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html. During evaluation, though we will be running the model in a forward pass over the entire validation set, so we can report the accuracies. The function `get_random_batch()` draws a random batch from the training/validation split (returning the names, labels and their tensor representations). In contrast, `get_all_batches()` returns all the batches.  

In [89]:
def get_random_batch(
    category_lines, batch_size, all_categories, all_letters, n_letters
):
    categories = random.choices(
        all_categories, k=batch_size
    )  # Selects batch_size random languages
    lines = [random.choice(category_lines[category]) for category in categories]
    category_tensors = torch.LongTensor(
        [all_categories.index(category) for category in categories]
    )
    line_tensors = line_to_tensor(
        batch_size, max_seq_length, lines, all_letters, n_letters
    )
    return categories, lines, category_tensors.to(device), line_tensors.to(device)


def get_all_batches(
    category_lines,
    all_categories,
    all_letters,
    n_letters,
    batch_size,
    max_seq_length
):
    all_lines = [(k, x) for k, l in category_lines.items() for x in l]
    num_samples = len(all_lines)
    batched_samples = [
        all_lines[i : i + batch_size] for i in range(0, num_samples, batch_size)
    ]
    for batch in batched_samples:
        categories, lines = map(list, zip(*batch))
        line_tensors = line_to_tensor(
            batch_size, max_seq_length, lines, all_letters, n_letters
        )
        category_tensors = torch.LongTensor(
            [all_categories.index(category) for category in categories]
        )
        yield categories, lines, category_tensors.to(device), line_tensors.to(device)

## Training/Evaluation Cycle 

The training and the evaluation functions `train()` and `evaluate()` are defined below. `train()` trains the model on a single batch and `evaluate()` runs the model on a single batch in a forward pass. During the training loop, the per-example gradients are computed and the parameters are updated subsequent to gradient clipping (to bound their sensitivity) and addition of noise.  

In [90]:
def train(rnn, criterion, optimizer, category_tensors, line_tensors):
    rnn.zero_grad()
    hidden = rnn.init_hidden()
    if isinstance(hidden, tuple):
        hidden = (hidden[0].to(device), hidden[1].to(device))
    else:
        hidden = hidden.to(device)
    output = rnn(line_tensors, hidden)
    loss = criterion(output, category_tensors)
    loss.backward()

    optimizer.step()

    return output, loss.data.item()


def evaluate(line_tensors, rnn):
    rnn.zero_grad()
    hidden = rnn.init_hidden()
    if isinstance(hidden, tuple):
        hidden = (hidden[0].to(device), hidden[1].to(device))
    else:
        hidden = hidden.to(device)
    output = rnn(line_tensors, hidden)
    return output

We also define `get_eval_metrics()` for metric computation, and a helper function `category_from_output()` which extracts the highest scoring language category at the ouput of the model.

In [91]:
def get_eval_metrics(
    rnn,
    category_lines,
    all_categories,
    all_letters,
    n_letters,
    batch_size,
    max_seq_length,
):
    pred = []
    truth = []
    for categories, _, _, line_tensors in get_all_batches(
        category_lines,
        all_categories,
        all_letters,
        n_letters,
        batch_size,
        max_seq_length,
    ):
        eval_output = evaluate(line_tensors, rnn)
        guess, _ = category_from_output(eval_output, all_categories)
        pred.extend(guess)
        truth.extend(categories)
    pred = pred[: min(len(pred), len(truth))]
    truth = truth[: min(len(pred), len(truth))]
    return balanced_accuracy_score(truth, pred)

def category_from_output(output, all_categories):
    top_n, top_i = output.data.topk(1)  # Tensor out of Variable with .data
    category_i = top_i.flatten()
    return [all_categories[category] for category in category_i], category_i

## Hyper-parameters

There are two sets of hyper-parameters associated with this model. The first are hyper-parameters which we would expect in any machine learning training, such as the learning rate and batch size. The second set are related to the privacy engine, where for example we define the amount of noise added to the gradients (`noise_multiplier`), and the maximum L2 norm to which the per-sample gradients are clipped (`max_grad_norm`). 

In [92]:
# Training hyper-parameters
learning_rate = 2.0

# Privacy engine hyper-parameters
noise_multiplier = 1.0
max_grad_norm = 1.5
delta = 8e-5

## Model

We define the name classification model in the cell below. Note that it is a simple char-LSTM classifier, where the input characters are passed through an `nn.Embedding` layer, and are subsequently input to the DPLSTM. Also note that the batch dimension is second in all tensors, and so we run the DPLSTM in the `batch_first=False` setting. We make sure that all intermediate activations in the network also maintain this batch ordering.

In [93]:
import torch
from torch import nn
from opacus.layers import DPLSTM

class CharNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_letters, batch_size):
        super(CharNNClassifier, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size

        self.embedding = nn.Embedding(n_letters, input_size)
        self.lstm = DPLSTM(input_size, hidden_size, batch_first=False)
        self.out_layer = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        input_emb = self.embedding(input)
        lstm_out, _ = self.lstm(input_emb, hidden)
        output = self.out_layer(lstm_out[-1].unsqueeze(0))
        return output[-1]

    def init_hidden(self):
        return (
            torch.zeros(1, self.batch_size, self.hidden_size),
            torch.zeros(1, self.batch_size, self.hidden_size),
        )

We now proceed to instantiate the objects (privacy engine, model and optimizer) for our differentially-private LSTM training.  However, the `nn.LSTM` is replaced with a `DPLSTM` module which enables us to calculate per-example gradients. The DPLSTM however, is limited to the following functionality (this example has uses supported values of the parameters):
1. A bias term must be present (`bias` flag set to True)
2. Single directional (`bidirectional` set to False)
3. Single layer (`num_layers` set to 1)
4. No dropout supported yet
5. Initial LSTM states (`h` and `c`) set to zero

In [94]:
# Set the device to run on a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define classifier parameters
n_hidden = 128  # Number of neurons in hidden layer after LSTM

rnn = CharNNClassifier(
    n_letters, n_hidden, n_categories, n_letters, batch_size
).to(device)

## Defining the privacy engine, optimizer and loss criterion for the problem

In [95]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

In [96]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine(
    rnn,
    batch_size=batch_size,
    sample_size=dataset_size,
    alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm,
    batch_first=False,
)
privacy_engine.attach(optimizer)

## Training the name classifier with privacy

Finally we can start training ! We will be training for 1000 iterations (where each iteration corresponds to a single batch of data). We will be reporting the privacy epsilon every `print_interval` iterations. We have also benchmarked this differentially-private model against a model without privacy and obtain almost identical performance. Further, the private model trained with Opacus incurs only minimal overhead in training time, with the differentially-private classifier only slightly slower (by a couple of minutes) than the non-private model.

In [97]:
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score

num_iterations = 1000
print_interval = 100

current_loss = 0

for iteration in tqdm(range(1, num_iterations + 1)):
    # Get a random training input and target batch
    _, _, category_tensors, line_tensors = get_random_batch(
        category_lines_train,
        batch_size,
        all_categories,
        all_letters,
        n_letters,
    )
    output, loss = train(
        rnn, criterion, optimizer, category_tensors, line_tensors
    )
    current_loss += loss

    # Print iteration number, loss, name and guess
    if iteration % print_every == 0:
        acc = get_eval_metrics(
            rnn,
            category_lines_eval,
            all_categories,
            all_letters,
            n_letters,
            batch_size,
            max_seq_length
        )
        epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(
            delta
        )
        print(
            f"Iteration={iteration} / Loss={loss:.4f} / "
            f"Eval Accuracy:{acc*100:.2f} / "
            f"Ɛ = {epsilon:.2f}, 𝛿 = {delta:.2f}) for α = {best_alpha:.2f}"
        )

 10%|█         | 100/1000 [00:20<09:33,  1.57it/s]

Iteration=100 / Loss=2.5397 / Eval Accuracy:10.82 / Ɛ = 4.10, 𝛿 = 0.00) for α = 4.60


 20%|██        | 201/1000 [00:40<04:51,  2.74it/s]

Iteration=200 / Loss=1.9222 / Eval Accuracy:30.25 / Ɛ = 5.39, 𝛿 = 0.00) for α = 4.00


 30%|███       | 301/1000 [00:57<03:33,  3.28it/s]

...

Iteration=800 / Loss=1.4360 / Eval Accuracy:48.69 / Ɛ = 10.43, 𝛿 = 0.00) for α = 2.90


 90%|█████████ | 900/1000 [02:27<00:35,  2.85it/s]

Iteration=900 / Loss=1.1599 / Eval Accuracy:49.11 / Ɛ = 11.10, 𝛿 = 0.00) for α = 2.80


100%|██████████| 1000/1000 [02:43<00:00,  6.12it/s]

Iteration=1000 / Loss=1.3244 / Eval Accuracy:50.65 / Ɛ = 11.74, 𝛿 = 0.00) for α = 2.70


The differentially-private name classification model obtains an accuracy of 50.65 with an epsilon of 11.74. This shows that we can achieve a good accuracy on this task, with minimal loss of privacy.

## Training the name classifier without privacy

 We also run a comparison with a non-private model to see if the performance obtained with privacy is comparable to it. To do this, we keep the parameters such as learning rate and batch size the same, and only define a different instance of the model along with a separate optimizer.

In [98]:
rnn_nodp = CharNNClassifier(
    n_letters, n_hidden, n_categories, n_letters, batch_size
).to(device)

optimizer_nodp = torch.optim.SGD(rnn_nodp.parameters(), lr=2.0)

In [99]:

current_loss = 0

for iteration in tqdm(range(1, num_iterations + 1)):
    # Get a random training input and target batch
    _, _, category_tensors, line_tensors = get_random_batch(
        category_lines_train,
        batch_size,
        all_categories,
        all_letters,
        n_letters,
    )
    output, loss = train(
        rnn_nodp, criterion, optimizer_nodp, category_tensors, line_tensors
    )
    current_loss += loss

    # Print iteration number, loss, name and guess
    if iteration % print_every == 0:
        acc = get_eval_metrics(
            rnn_nodp,
            category_lines_eval,
            all_categories,
            all_letters,
            n_letters,
            batch_size,
            max_seq_length
        )
        epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(
            delta
        )
        print(
            f"Iteration={iteration} / Loss={loss:.4f} / "
            f"Eval Accuracy:{acc*100:.2f}"
        )

 10%|█         | 101/1000 [00:10<02:33,  5.87it/s]

Iteration=100 / Loss=1.9366 / Eval Accuracy:34.36


 20%|██        | 201/1000 [00:22<04:13,  3.15it/s]

Iteration=200 / Loss=1.3896 / Eval Accuracy:48.08


 30%|███       | 301/1000 [00:34<01:51,  6.26it/s]

...

Iteration=800 / Loss=0.2735 / Eval Accuracy:54.72


 90%|█████████ | 901/1000 [01:36<00:18,  5.49it/s]

Iteration=900 / Loss=0.2580 / Eval Accuracy:54.86


100%|██████████| 1000/1000 [01:46<00:00,  9.43it/s]

Iteration=1000 / Loss=0.2827 / Eval Accuracy:52.29


We run the training loop again, this time without privacy and for the same number of iterations. 

The non-private classifier obtains an accuracy of 52.29 with the same parameters and number of epochs. While we are effectively trading off performance on the name classification task for a lower loss of privacy, the difference in accuracy is only around 1.64% at a very low epsilon of 11.74. 